In [ ]:
from datasets import load_dataset

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
ds_train = load_dataset("MKZuziak/ISIC_2019_224", split="train") # que le split train disponible

In [3]:
print(ds_train.features)

{'image': Image(mode=None, decode=True), 'label': Value('string')}


In [30]:
from utils import ISICDataset

my_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# On passe le dataset chargé précédemment à notre classe
custom_ds = ISICDataset(ds_train, transform=my_transform)

train_loader = DataLoader(
    custom_ds, 
    batch_size=16, 
    shuffle=True, # Important pour le déséquilibre
)

# --- ÉTAPE 3 : Affichage des 16 premières images ---
# On récupère un batch

batch = next(iter(train_loader))
images, labels = batch

# plt.figure(figsize=(8, 8))
# for i in range(16):
#     plt.subplot(4, 4, i + 1)
#     # PyTorch utilise (C, H, W), Matplotlib attend (H, W, C) -> on utilise permute
#     img_display = images[i].permute(1, 2, 0).numpy()
#     plt.imshow(img_display)
#     plt.title(f"Classe: {labels[i].item()}")
#     plt.axis("off")

# plt.tight_layout()
# plt.show()

TypeError: '_SingleProcessDataLoaderIter' object is not callable

In [31]:
from model import cnn_model
from train import setup_training, train_one_epoch

In [33]:
# Init the training
model = cnn_model(num_classes=8)
criterion, optimizer = setup_training(model, learning_rate=0.01, momentum=0.9)

DEVICE = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(DEVICE)
model = model.to(DEVICE)
# print(model)

train_losses = []
valid_losses = []

EPOCHS = 10
# loop over the dataset multiple times
for epoch in range(EPOCHS):
    model.train()
    train_epoch_loss = train_one_epoch(model, criterion, optimizer, train_loader, DEVICE)
    # model.eval()
    # valid_epoch_loss = valid_one_epoch(model, val_loader)

    print(f"EPOCH={epoch}, TRAIN={train_epoch_loss:.05f}")

    train_losses.append(train_epoch_loss)
    # valid_losses.append(valid_epoch_loss)

cpu


  0%|          | 1/1584 [00:05<2:21:57,  5.38s/it]

Batch 0, curr loss = 2.091


  1%|          | 11/1584 [01:00<2:22:59,  5.45s/it]

Batch 10, curr loss = 1.702


...

Batch 20, curr loss = 1.889


  2%|▏         | 31/1584 [02:49<2:22:42,  5.51s/it]

Batch 30, curr loss = 1.889


  2%|▏         | 35/1584 [03:12<2:21:49,  5.49s/it]


KeyboardInterrupt: 